In [12]:
#import needed libraries
import tqdm
import pickle
import pandas as pd
import datetime
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [13]:
# connect to neo4j
df_pw = pd.read_csv("pw.csv", delimiter=",")
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [4]:
# get reviews
berlusconi_reviews  = pd.read_csv('reviews_price_btc.csv')
berlusconi_reviews.time = pd.to_datetime(berlusconi_reviews.time)

In [6]:
#Used for manual analysis
# reviews_of_identified_payoutTAs = berlusconi_reviews [berlusconi_reviews ["vendor_review_id"]==107526]
# reviews_of_identified_payoutTAs = reviews_of_identified_payoutTAs.append(berlusconi_reviews [berlusconi_reviews ["vendor_review_id"]==106746])
# reviews_of_identified_payoutTAs = reviews_of_identified_payoutTAs.append(berlusconi_reviews [berlusconi_reviews ["vendor_review_id"]==106732])
# reviews_of_identified_payoutTAs = reviews_of_identified_payoutTAs.append(berlusconi_reviews [berlusconi_reviews ["vendor_review_id"]==106762])
# reviews_of_identified_payoutTAs = reviews_of_identified_payoutTAs.append(berlusconi_reviews [berlusconi_reviews ["vendor_review_id"]==106687])

# display(reviews_of_identified_payoutTAs)

In [ ]:
# get all transactions matching the berlusconi pattern and a review
transaction_t2 = pd.DataFrame()
berlusconi_reviews2 = berlusconi_reviews 
for i in berlusconi_reviews .itertuples():
    neo_results = []
    neo_results2 = []
    neo_results3 = []

    # time frame in which payout transaction may have occured (start <= payout.date <= end)
    # most identified payout TAs occur 2 days after the review. 
    # To reduce number of matches setting "start = end = reviewdate + 2 days" might be a good idea
    start = str(i.time.date())
    end = str(i.time.date() + timedelta(days=2))
    # range of possible inSum values of the payout TA (could also implement the same for outSum, if required)
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi*2.0,0))
    

    #neo_query with all possible parameters. Remove whatever you don't want to use
    neo_query = session.run("""
    WITH
    '""" + start + """' as startDate,
    '""" + end + """' as endDate,
    """+ minSum + """ as minSum, 
    """ + maxSum + """ as maxSum 
    MATCH (t:Transaction)<-[:SENDS]-(a:Address)
    WHERE date(startDate) <= t.date <= date(endDate)
        and t.outDegree = 2
        and minSum <= t.inSum < maxSum
        and (a.address STARTS WITH "3" OR a.address STARTS WITH "bc1")
    WITH t, minSum
    MATCH (e0:Address)-[s0:SENDS]->(t0:Transaction)-[r0:RECEIVES]->(e1:Address)-[s1:SENDS]->(t)
    WHERE e1.address STARTS WITH "3"
    AND t0.outDegree=2
    AND (e0.address STARTS WITH "3" OR e0.address STARTS WITH "bc1")
    AND 900000<s0.value< 1100000
    AND 900000<r0.value< 1100000
    AND 900000<s1.value< 1100000
    AND r0.value=s1.value
    WITH t, minSum
    MATCH (t)-[r:RECEIVES]->(change:Address)-[s:SENDS]->(t3:Transaction)-[r3:RECEIVES]->()
    WHERE (change.address STARTS WITH "3" OR change.address STARTS WITH "bc1")
        AND change.inDegree = 1
        AND r.value = s.value
        AND 900000<s.value< 1100000
        AND t3.outDegree=2
    WITH t, minSum, change
    MATCH (t)-[r:RECEIVES]->(vendor:Address)
    WHERE minSum <= r.value < minSum * 1.05 
    AND vendor <> change
    RETURN DISTINCT t, r.value, vendor.address
    """)
    
    for r in neo_query:
        neo_results.append(r[0])
        neo_results2.append(r[1])
        neo_results3.append(r[2])

    
    df_neo4j = pd.DataFrame(neo_results)
    df_neo4j["vendor"] = i.vendor_id
    df_neo4j["review"] = i.vendor_review_id
    df_neo4j["rvalue"] = pd.Series(neo_results2)
    df_neo4j["vendor_address"] = pd.Series(neo_results3)
    transaction_t2 = pd.concat([transaction_t2,df_neo4j])
    
display(transaction_t2)
transaction_t2.to_csv('transaction_t2.csv')

/tmp/ipykernel_441537/3407464277.py:72: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j["rvalue"] = pd.Series(neo_results2)
/tmp/ipykernel_441537/3407464277.py:73: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j["vendor_address"] = pd.Series(neo_results3)


In [17]:
# how many address-review matches are there now?
len(transaction_t2)

1981870

In [32]:
# exclude the transactions for which there exist a pay to public key sending address

tx_list=list(transaction_t2['txid'])
neo_results = []
neo_results2 = []
neo_query = session.run("""MATCH (t:Transaction)<-[:SENDS]-(a:Address)
    WHERE t.txid in """+ str(tx_list) +"""
    and a.address STARTS WITH "1"
return t, a.address""")

for r in neo_query:
        neo_results.append(r[0])
        neo_results2.append(r[1])
df_neo4j = pd.DataFrame(neo_results)
df_neo4j["address"] = pd.Series(neo_results2)

# dataframe with only correct transactions
direct_result=transaction_t2[~transaction_t2['txid'].isin(df_neo4j['txid'])]
direct_result.to_csv('direct_results.csv')